In [ ]:
import csv

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

matplotlib.rcParams.update({'font.size': 15})

# Select spectrograms with annotations picked by a given number of annotator

In [ ]:
# paths to spectrograms
path_of_datasets_oha = "PATH_TO_SPECTROGRAMS"
path_of_datasets_hyd = "PATH_TO_SPECTROGRAMS"  

DELTA_S = 50

nb_annotators = 1

In [ ]:
lines = []

with open(f'{path_of_datasets_oha}/dataset.csv', "r") as f:
    csv_reader = csv.reader(f, delimiter=",")
    lines_oha = list(csv_reader)    
        

with open(f'{path_of_datasets_hyd}/dataset.csv', "r") as f:
    csv_reader = csv.reader(f, delimiter=",")
    lines_hyd = list(csv_reader)
    
for line in lines_oha + lines_hyd:
    if line[1] == "positive" and np.all(np.array(line[3::2], dtype=np.int32) == nb_annotators):  # if all annotations were seen by n annotator
            lines.append(line)
print(len(lines), "files found")

np.random.seed(5)
np.random.shuffle(lines)

In [ ]:
nl, nc = 3, 3
plt.subplots(nc, nl, figsize=(6*nc, 4*nl))

for l in range(nl):
    for c in range(nc):
        idx = l*nc + c
        ax = plt.subplot(nc, nl, 1+idx)
        
        img = mpimg.imread(lines[idx][0])
        ax.imshow(img, cmap="jet", extent=(-DELTA_S, DELTA_S, 0, 120), aspect=0.5, vmin=0, vmax=1)
        for pick in lines[idx][2::2]:
            ax.arrow(float(pick), 111, 0, -6,
                                    length_includes_head=True, head_width=5, head_length=15, color="red")
        
        plt.ylabel("Frequency (Hz)")
        plt.xlabel("Time difference from center (s)")
        
plt.savefig(f"../../../../../data/figures/example_spectrograms_{nb_annotators}-annotator.png")

# Select spectrograms with annotations picked by a given number of annotators and missed by all classifiers

In [ ]:
paths_of_picks_TiSSNet = "../../../../../data/npy/TiSSNet"
paths_of_picks_acousticPhaseNet = "../../../../../data/npy/AcousticPhaseNet"
paths_of_picks_sgbt20 = "../../../../../data/npy/SGBT20"
paths_of_picks_sgbt100 = "../../../../../data/npy/SGBT100"
paths_of_picks_resnet = "../../../../../data/npy/ResNet-50"

fn_paths = [paths_of_picks_TiSSNet, paths_of_picks_acousticPhaseNet, paths_of_picks_sgbt20, paths_of_picks_sgbt100, paths_of_picks_resnet]

In [ ]:
scores_seg = [[np.load(f'{path}/{dataset}/TP.npy', allow_pickle=True) for dataset in ("OHASISBIO-1", "HYDROMOMAR-1")] for path in fn_paths][:2]
scores_class = [[np.load(f'{path}/{dataset}/FN.npy', allow_pickle=True) for dataset in ("OHASISBIO-1", "HYDROMOMAR-1")] for path in fn_paths][2:]

# browse all detections of segmentation models and apply a threshold of 0.5 to decide if the detection is positive or negative (to make it comparable to the FN array of other models)
fn_idxs = []
for i, m in enumerate(scores_seg):
    fn_idxs.append([[], []])
    for idx in m[0].item().keys():
        if m[0].item()[idx] < 0.5:
            fn_idxs[i][0].append(idx)
    for idx in m[1].item().keys():
        if m[1].item()[idx] < 0.5:
            fn_idxs[i][1].append(idx)

fn_idxs.extend([list(s) for s in scores_class])

In [ ]:
idx_oha = [d[0] for d in fn_idxs]
common_fn_oha = set.intersection(*map(set,idx_oha))  # intersection of missed events
common_fn_oha = [lines_oha[idx] for idx in common_fn_oha]

idx_hyd = [d[1] for d in fn_idxs]
common_fn_hyd = set.intersection(*map(set,idx_hyd))  # intersection of missed events
common_fn_hyd = [lines_hyd[idx] for idx in common_fn_hyd]

nb_annotators = 3

common_fn = common_fn_oha + common_fn_hyd
kept_fn = []
for line in common_fn:
    if np.all(np.array(line[3::2], dtype=np.int32) == nb_annotators):  # if all annotations were seen by n annotator
            kept_fn.append(line)
np.random.seed(5)
np.random.shuffle(kept_fn)

print(f"{len(kept_fn)} common false negatives were kept")

In [ ]:
nl, nc = 3, 3
plt.subplots(nc, nl, figsize=(6*nc, 4*nl))

for l in range(nl):
    for c in range(nc):
        idx = l*nc + c
        ax = plt.subplot(nc, nl, 1+idx)
        
        img = mpimg.imread(kept_fn[idx][0])
        ax.imshow(img, cmap="jet", extent=(-DELTA_S, DELTA_S, 0, 120), aspect=0.5, vmin=0, vmax=1)
        
        plt.ylabel("Frequency (Hz)")
        plt.xlabel("Time difference from center (s)")
        
plt.savefig(f"../../../../../data/figures/example_spectrograms_FN_{nb_annotators}-annotator.png")